In [4]:
import cv2
import glob
from shutil import copyfile
import os
import matplotlib.pyplot as plt
import numpy as np
import random
import re
import pickle

%matplotlib inline

In [2]:
# clear all folders
def clear_folders(folder_name):
    emo_folders = glob.glob("%s/*" % folder_name)
    for emo_folder in emo_folders:
        # grab all image files and delete them
        images = glob.glob("%s/*"%emo_folder)
        print(emo_folder, len(images))
        for image in images:
            try:
                if os.path.isfile(image):
                    os.unlink(image)
            except Exception as e:
                print(e)
                return -1
    return 1
        

In [21]:
face_encoding = {0:"neutral", 1:"anger", 2:"contempt", 3:"disgust", 4:"fear", 5:"happy", 6:"sadness", 7:"surprise"}
# emotions = ["neutral", "anger", "contempt", "disgust", "fear", "happy", "sadness", "surprise"] #Define emotion order
emotions = ["neutral", "anger", "disgust", "happy", "surprise"]
participants = glob.glob("ck_export/source_emotion/*") #Returns a list of all folders with participant numbers




In [ ]:
for x in participants:
    part = "%s" %x[-4:] # store current participant number
    for sessions in glob.glob("%s/*"%x): # Store list of sessions for current participant
        for files in glob.glob("%s/*" %sessions):
            current_session = files[30:-30]
            file = open(files, 'r')
            
            emotion = int(float(file.readline())) # emotions are encoded as float, readline as float then convert to integer.
            sourcefile_emotion = glob.glob("ck_export/source_images/%s/%s/*" % (part, current_session))[-1] # get path for last image in sequence, which containes the emotion
            sourcefile_neutral = glob.glob("ck_export/source_images/%s/%s/*" % (part, current_session))[0] # do same for neutral image
            
            dest_neut = "ck_export/sorted_set/neutral/%s" %sourcefile_neutral[35:] # generate path to put neutral image
            dest_emot = "ck_export/sorted_set/%s/%s" % (emotions[emotion], sourcefile_emotion[35:]) # Do the same for emotion containing image
            
            copyfile(sourcefile_neutral, dest_neut) # Copy file
            copyfile(sourcefile_emotion, dest_emot) # Copy file

In [53]:
print("len of participants:", len(participants))
for participant in participants:
    part = "%s"%participant[-4:]
    for session in glob.glob("%s/*"%participant):
        # now in a specific session
        current_session = session[-3:]
        for file in glob.glob("%s/*"%session):
            with open(file, 'r') as emo_file:
                emotion = int(float(emo_file.readline()))
            sourcefile_emotion = sorted(glob.glob("ck_export/source_images/%s/%s/*"%(part, current_session)))
            sourcefile_neutral = sorted(glob.glob("ck_export/source_images/%s/%s/*"%(part, current_session)))
            ses = sourcefile_emotion[-3:]
            sn = sourcefile_neutral[0]
            
            dest_n = "ck_export/sorted_set/neutral/%s"%sn[33:]
            dest_es = ["ck_export/sorted_set/%s/%s" % (emotions[emotion], se[33:]) for se in ses]
            for i in range(3):
                copyfile(ses[i], dest_es[i])
            copyfile(sn, dest_n)
#             if emotion == 1:
#                 plt.imshow(cv2.cvtColor(cv2.imread(sourcefile_emotion[0]), cv2.COLOR_BGR2RGB))
#                 plt.show()

len of participants: 123


In [ ]:
print(se1)
print(se1[33:])

In [52]:
clear_folders("ck_export/sorted_set")
print(sourcefile_emotion)
print(sourcefile_emotion[35:])
print(current_session)

ck_export/sorted_set/happy 207
ck_export/sorted_set/sadness 84
ck_export/sorted_set/neutral 327
ck_export/sorted_set/disgust 177
ck_export/sorted_set/anger 135
ck_export/sorted_set/contempt 54
ck_export/sorted_set/surprise 249
ck_export/sorted_set/fear 75
['ck_export/source_images/S506/002/S506_002_00000001.png', 'ck_export/source_images/S506/002/S506_002_00000002.png', 'ck_export/source_images/S506/002/S506_002_00000003.png', 'ck_export/source_images/S506/002/S506_002_00000004.png', 'ck_export/source_images/S506/002/S506_002_00000005.png', 'ck_export/source_images/S506/002/S506_002_00000006.png', 'ck_export/source_images/S506/002/S506_002_00000007.png', 'ck_export/source_images/S506/002/S506_002_00000008.png', 'ck_export/source_images/S506/002/S506_002_00000009.png']
[]
002


In [60]:
faceDet = cv2.CascadeClassifier("OpenCV_FaceCascade/haarcascade_frontalface_default.xml")
faceDet_two = cv2.CascadeClassifier("OpenCV_FaceCascade/haarcascade_frontalface_alt2.xml")
faceDet_three = cv2.CascadeClassifier("OpenCV_FaceCascade/haarcascade_frontalface_alt.xml")
faceDet_four = cv2.CascadeClassifier("OpenCV_FaceCascade/haarcascade_frontalface_alt_tree.xml")

def detect_faces(emotion):
    files = sorted(glob.glob("ck_export/sorted_set/%s/*" %emotion)) #Get list of all images with emotion

    filenumber = 0
    for f in files:
        frame = cv2.imread(f) #Open image
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #Convert image to grayscale
        
        #Detect face using 4 different classifiers
        face = faceDet.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face_two = faceDet_two.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face_three = faceDet_three.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)
        face_four = faceDet_four.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=10, minSize=(5, 5), flags=cv2.CASCADE_SCALE_IMAGE)

        #Go over detected faces, stop at first detected face, return empty if no face.
        if len(face) == 1:
            facefeatures = face
        elif len(face_two) == 1:
            facefeatures = face_two
        elif len(face_three) == 1:
            facefeatures = face_three
        elif len(face_four) == 1:
            facefeatures = face_four
        else:
            facefeatures = ""
        
        #Cut and save face
        for (x, y, w, h) in facefeatures: #get coordinates and size of rectangle containing face
#             print("face found in file: %s" %f)
            gray = gray[y:y+h, x:x+w] #Cut the frame to size
            
            try:
                out = cv2.resize(gray, (350, 350)) #Resize face so all images have same size
                cv2.imwrite("ck_export/dataset/%s/%s.jpg" %(emotion, filenumber), out) #Write image
            except:
               pass #If error, pass file
        filenumber += 1 #Increment image number

for emotion in emotions: 
    detect_faces(emotion) #Call functiona

face found in file: ck_export/sorted_set/neutral/S088_004_00000001.png
face found in file: ck_export/sorted_set/neutral/S136_003_00000001.png
face found in file: ck_export/sorted_set/neutral/S014_002_00000001.png
face found in file: ck_export/sorted_set/neutral/S115_001_00000001.png
face found in file: ck_export/sorted_set/neutral/S131_010_00000001.png
face found in file: ck_export/sorted_set/neutral/S032_001_00000001.png
face found in file: ck_export/sorted_set/neutral/S115_004_00000001.png
face found in file: ck_export/sorted_set/neutral/S147_002_00000001.png
face found in file: ck_export/sorted_set/neutral/S014_003_00000001.png
face found in file: ck_export/sorted_set/neutral/S034_005_00000001.png
face found in file: ck_export/sorted_set/neutral/S089_003_00000001.png
face found in file: ck_export/sorted_set/neutral/S026_006_00000001.png
face found in file: ck_export/sorted_set/neutral/S011_006_00000001.png
face found in file: ck_export/sorted_set/neutral/S134_003_00000001.png
face f

face found in file: ck_export/sorted_set/neutral/S066_002_00000001.png
face found in file: ck_export/sorted_set/neutral/S155_002_00000001.png
face found in file: ck_export/sorted_set/neutral/S113_008_00000001.png
face found in file: ck_export/sorted_set/neutral/S895_002_00000001.png
face found in file: ck_export/sorted_set/neutral/S113_001_00000001.png
face found in file: ck_export/sorted_set/neutral/S098_004_00000001.png
face found in file: ck_export/sorted_set/neutral/S037_001_00000001.png
face found in file: ck_export/sorted_set/neutral/S106_002_00000001.png
face found in file: ck_export/sorted_set/neutral/S050_001_00000001.png
face found in file: ck_export/sorted_set/neutral/S135_012_00000001.png
face found in file: ck_export/sorted_set/neutral/S137_001_00000001.png
face found in file: ck_export/sorted_set/neutral/S057_006_00000001.png
face found in file: ck_export/sorted_set/neutral/S082_001_00000001.png
face found in file: ck_export/sorted_set/neutral/S107_005_00000001.png
face f

face found in file: ck_export/sorted_set/neutral/S095_001_00000001.png
face found in file: ck_export/sorted_set/neutral/S087_004_00000001.png
face found in file: ck_export/sorted_set/neutral/S056_004_00000001.png
face found in file: ck_export/sorted_set/neutral/S124_003_00000001.png
face found in file: ck_export/sorted_set/neutral/S505_002_00000001.png
face found in file: ck_export/sorted_set/neutral/S087_005_00000001.png
face found in file: ck_export/sorted_set/neutral/S046_004_00000001.png
face found in file: ck_export/sorted_set/neutral/S125_007_00000001.png
face found in file: ck_export/sorted_set/neutral/S081_001_00000001.png
face found in file: ck_export/sorted_set/neutral/S090_007_00000001.png
face found in file: ck_export/sorted_set/neutral/S098_003_00000001.png
face found in file: ck_export/sorted_set/neutral/S078_001_00000001.png
face found in file: ck_export/sorted_set/neutral/S055_006_00000001.png
face found in file: ck_export/sorted_set/neutral/S073_001_00000001.png
face f

face found in file: ck_export/sorted_set/anger/S089_003_00000034.png
face found in file: ck_export/sorted_set/anger/S032_003_00000015.png
face found in file: ck_export/sorted_set/anger/S127_010_00000017.png
face found in file: ck_export/sorted_set/anger/S501_001_00000067.png
face found in file: ck_export/sorted_set/anger/S506_001_00000039.png
face found in file: ck_export/sorted_set/anger/S022_005_00000032.png
face found in file: ck_export/sorted_set/anger/S082_005_00000015.png
face found in file: ck_export/sorted_set/anger/S042_004_00000020.png
face found in file: ck_export/sorted_set/anger/S090_007_00000012.png
face found in file: ck_export/sorted_set/anger/S042_004_00000018.png
face found in file: ck_export/sorted_set/anger/S506_001_00000038.png
face found in file: ck_export/sorted_set/anger/S111_006_00000010.png
face found in file: ck_export/sorted_set/anger/S504_001_00000021.png
face found in file: ck_export/sorted_set/anger/S999_001_00000017.png
face found in file: ck_export/sort

face found in file: ck_export/sorted_set/contempt/S505_002_00000021.png
face found in file: ck_export/sorted_set/contempt/S503_002_00000008.png
face found in file: ck_export/sorted_set/contempt/S503_002_00000006.png
face found in file: ck_export/sorted_set/contempt/S151_002_00000029.png
face found in file: ck_export/sorted_set/contempt/S506_002_00000009.png
face found in file: ck_export/sorted_set/contempt/S149_002_00000013.png
face found in file: ck_export/sorted_set/contempt/S148_002_00000015.png
face found in file: ck_export/sorted_set/contempt/S158_002_00000011.png
face found in file: ck_export/sorted_set/contempt/S157_002_00000009.png
face found in file: ck_export/sorted_set/contempt/S147_002_00000013.png
face found in file: ck_export/sorted_set/contempt/S151_002_00000028.png
face found in file: ck_export/sorted_set/contempt/S503_002_00000007.png
face found in file: ck_export/sorted_set/contempt/S154_002_00000012.png
face found in file: ck_export/sorted_set/contempt/S895_002_00000

...

face found in file: ck_export/sorted_set/happy/S098_004_00000014.png
face found in file: ck_export/sorted_set/happy/S115_008_00000016.png
face found in file: ck_export/sorted_set/happy/S032_006_00000016.png
face found in file: ck_export/sorted_set/happy/S087_005_00000010.png
face found in file: ck_export/sorted_set/happy/S096_004_00000009.png
face found in file: ck_export/sorted_set/happy/S075_006_00000023.png
face found in file: ck_export/sorted_set/happy/S062_004_00000023.png
face found in file: ck_export/sorted_set/happy/S044_003_00000012.png
face found in file: ck_export/sorted_set/happy/S100_006_00000016.png
face found in file: ck_export/sorted_set/happy/S089_002_00000020.png
face found in file: ck_export/sorted_set/happy/S053_004_00000023.png
face found in file: ck_export/sorted_set/happy/S109_006_00000014.png
face found in file: ck_export/sorted_set/happy/S114_006_00000021.png
face found in file: ck_export/sorted_set/happy/S064_003_00000025.png
face found in file: ck_export/sort

face found in file: ck_export/sorted_set/happy/S026_006_00000011.png
face found in file: ck_export/sorted_set/happy/S042_006_00000017.png
face found in file: ck_export/sorted_set/happy/S083_003_00000018.png
face found in file: ck_export/sorted_set/happy/S115_008_00000017.png
face found in file: ck_export/sorted_set/happy/S066_003_00000012.png
face found in file: ck_export/sorted_set/happy/S125_005_00000011.png
face found in file: ck_export/sorted_set/happy/S074_005_00000043.png
face found in file: ck_export/sorted_set/happy/S095_007_00000020.png
face found in file: ck_export/sorted_set/happy/S094_004_00000012.png
face found in file: ck_export/sorted_set/happy/S093_004_00000014.png
face found in file: ck_export/sorted_set/happy/S097_006_00000017.png
face found in file: ck_export/sorted_set/happy/S091_003_00000019.png
face found in file: ck_export/sorted_set/happy/S072_006_00000022.png
face found in file: ck_export/sorted_set/happy/S042_006_00000015.png
face found in file: ck_export/sort

face found in file: ck_export/sorted_set/sadness/S505_006_00000017.png
face found in file: ck_export/sorted_set/surprise/S052_001_00000013.png
face found in file: ck_export/sorted_set/surprise/S135_001_00000037.png
face found in file: ck_export/sorted_set/surprise/S124_001_00000012.png
face found in file: ck_export/sorted_set/surprise/S046_002_00000006.png
face found in file: ck_export/sorted_set/surprise/S122_001_00000012.png
face found in file: ck_export/sorted_set/surprise/S066_002_00000021.png
face found in file: ck_export/sorted_set/surprise/S086_001_00000018.png
face found in file: ck_export/sorted_set/surprise/S053_001_00000021.png
face found in file: ck_export/sorted_set/surprise/S056_003_00000010.png
face found in file: ck_export/sorted_set/surprise/S076_001_00000015.png
face found in file: ck_export/sorted_set/surprise/S074_002_00000015.png
face found in file: ck_export/sorted_set/surprise/S110_001_00000013.png
face found in file: ck_export/sorted_set/surprise/S099_001_000000

face found in file: ck_export/sorted_set/surprise/S055_001_00000010.png
face found in file: ck_export/sorted_set/surprise/S071_001_00000011.png
face found in file: ck_export/sorted_set/surprise/S102_002_00000017.png
face found in file: ck_export/sorted_set/surprise/S132_008_00000010.png
face found in file: ck_export/sorted_set/surprise/S125_007_00000009.png
face found in file: ck_export/sorted_set/surprise/S117_001_00000014.png
face found in file: ck_export/sorted_set/surprise/S087_001_00000009.png
face found in file: ck_export/sorted_set/surprise/S062_002_00000014.png
face found in file: ck_export/sorted_set/surprise/S022_001_00000030.png
face found in file: ck_export/sorted_set/surprise/S069_002_00000014.png
face found in file: ck_export/sorted_set/surprise/S138_004_00000012.png
face found in file: ck_export/sorted_set/surprise/S014_001_00000027.png
face found in file: ck_export/sorted_set/surprise/S035_001_00000015.png
face found in file: ck_export/sorted_set/surprise/S065_003_00000

face found in file: ck_export/sorted_set/surprise/S114_001_00000016.png
face found in file: ck_export/sorted_set/surprise/S111_001_00000012.png
face found in file: ck_export/sorted_set/surprise/S081_001_00000019.png
face found in file: ck_export/sorted_set/surprise/S077_001_00000028.png
face found in file: ck_export/sorted_set/surprise/S087_001_00000010.png
face found in file: ck_export/sorted_set/surprise/S111_001_00000013.png
face found in file: ck_export/sorted_set/surprise/S092_001_00000016.png
face found in file: ck_export/sorted_set/surprise/S044_001_00000022.png
face found in file: ck_export/sorted_set/surprise/S044_001_00000024.png
face found in file: ck_export/sorted_set/surprise/S079_001_00000011.png
face found in file: ck_export/sorted_set/surprise/S127_001_00000017.png
face found in file: ck_export/sorted_set/surprise/S022_001_00000029.png
face found in file: ck_export/sorted_set/surprise/S129_002_00000011.png
face found in file: ck_export/sorted_set/surprise/S132_008_00000

## Train test split

In [62]:
clear_folders("ck_export/dataset/train")
clear_folders("ck_export/dataset/test")

ck_export/dataset/train/happy 207
ck_export/dataset/train/sadness 84
ck_export/dataset/train/neutral 327
ck_export/dataset/train/disgust 177
ck_export/dataset/train/anger 135
ck_export/dataset/train/contempt 54
ck_export/dataset/train/surprise 249
ck_export/dataset/train/fear 75
ck_export/dataset/test/happy 207
ck_export/dataset/test/sadness 84
ck_export/dataset/test/neutral 327
ck_export/dataset/test/disgust 177
ck_export/dataset/test/anger 135
ck_export/dataset/test/contempt 54
ck_export/dataset/test/surprise 249
ck_export/dataset/test/fear 75


1

In [22]:
fishface = cv2.face.FisherFaceRecognizer_create()
data = {}
def train_test_split(emotion): #Define function to get file list, randomly shuffle it and split
    files = glob.glob("ck_export/dataset/%s/*"%emotion)
    random.shuffle(files)
    training = files[:int(len(files) * 0.8)] # get first 80% of the file list
    test = files[-int(len(files) * 0.2): ] # get the last 205 of the file list
    return training, test

def make_sets(export=False):
    training_data = []
    training_labels = []
    test_data = []
    test_labels = []
    for emotion in emotions:
        training, test = train_test_split(emotion)
        # Append data to training and test list, and generate labels 0-7
        for item in training:
            if export:
                copyfile(item, "ck_export/dataset/train/%s/%s"%(emotion, re.findall(r'[\d]*.jpg', item)[0]))
            image = cv2.imread(item) # open image
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert to grayscale
            training_data.append(gray) # append image to training data
            training_labels.append(emotions.index(emotion))
            
        for item in test:
            if export:
                copyfile(item, "ck_export/dataset/test/%s/%s"% (emotion, re.findall(r'[\d]*.jpg', item)[0]))
            image = cv2.imread(item)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            test_data.append(gray)
            test_labels.append(emotions.index(emotion))
    
    return training_data, training_labels, test_data, test_labels

def run_recognizer():
    training_data, training_labels, test_data, test_labels = make_sets()
    print("training fisher face classifier")
    print("size of training set is:", len(training_labels), "images")
    fishface.train(training_data, np.asarray(training_labels))
    
    print("predicting classification set")
    cnt=0
    correct=0
    incorrect=0
    for image in test_data:
        pred, conf = fishface.predict(image)
        if pred == test_labels[cnt]:
            correct += 1
            cnt += 1
        else:
            incorrect += 1
            cnt += 1
    return ((100*correct)/(correct + incorrect))



In [ ]:
# now run it
metascore = []
for i in range(0, 10):
    correct = run_recognizer()
    print("got", correct, "percent correct! ")
    metascore.append(correct)
    
print("\n\nend score:", np.mean(metascore), "percent correct !")

In [5]:
import pickle

with open("fisher face classifer.pickle", 'wb') as handle:
    pickle.dump(fishface, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [23]:
correct = run_recognizer()
print("got", correct, "percent correct! ")

training fisher face classifier
size of training set is: 874 images
predicting classification set
got 97.6958525345622 percent correct! 


In [9]:
image = cv2.imread("ck_export/dataset/test/anger/100.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
pred, conf = fishface.predict(gray)

In [10]:
print(pred)

1


In [ ]:
video_capture = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX

feelings_faces = []
for index, emotion in enumerate(emotions):
  feelings_faces.append(cv2.imread('./emojis/' + emotion + '.png', -1))

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    if frame is not None:
        # Predict result with network
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        gray = cv2.resize(gray, (350, 350))
        pred, conf = fishface.predict(gray)
        
        for index, emotion in enumerate(emotions):
            cv2.putText(frame, emotion, (10, index * 20 + 20), cv2.FONT_HERSHEY_PLAIN, 0.5, (0, 255, 0), 1);
#             cv2.rectangle(frame, (130, index * 20 + 10), (130 + int(pred * 100), (index + 1) * 20 + 4), (255, 0, 0), -1)
        
        face_image = feelings_faces[pred]

        # Ugly transparent fix
        for c in range(0, 3):
            frame[200:320, 10:130, c] = face_image[:,:,c] * (face_image[:, :, 3] / 255.0) +  frame[200:320, 10:130, c] * (1.0 - face_image[:, :, 3] / 255.0)

        cv2.imshow('Video', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

In [18]:
video_capture.release()
cv2.destroyAllWindows()